# Visualisation des bruits complexes

Ce notebook permet d'explorer l'influence du paramètre `noise_level` (de 0.0 à 2.0, par pas de 0.1) sur chacun des profils de bruit complexe disponibles dans `project.data.noise`. Pour chaque combinaison type/niveau, cinq réalisations indépendantes sont générées afin d'illustrer la variabilité statistique du procédé de synthèse.

## Préparation

On commence par importer les dépendances nécessaires et par définir un spectre synthétique propre qui servira de référence pour l'étude. Ce spectre propre reproduit grossièrement plusieurs raies d'absorption sur un continuum unitaire.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets

from project.data.noise import add_noise_variety

plt.rcParams['figure.figsize'] = (10, 4)
plt.rcParams['axes.grid'] = True


In [ ]:
torch.manual_seed(0)
np.random.seed(0)

spectral_length = 1024
x = torch.linspace(0.0, 1.0, spectral_length)

# Construction d'un spectre propre synthétique avec plusieurs raies d'absorption.
def build_clean_spectrum(x: torch.Tensor) -> torch.Tensor:
    centers = torch.tensor([0.18, 0.32, 0.58, 0.76])
    widths = torch.tensor([0.015, 0.030, 0.020, 0.025])
    depths = torch.tensor([0.35, 0.25, 0.40, 0.30])

    spectrum = torch.ones_like(x)
    for c, w, d in zip(centers, widths, depths):
        spectrum = spectrum - d * torch.exp(-0.5 * ((x - c) / w) ** 2)
    # Légère modulation pour imiter une pente instrumentale.
    spectrum = spectrum * (1.0 + 0.05 * torch.sin(2 * torch.pi * x))
    return spectrum

clean_reference = build_clean_spectrum(x)
# Baseline normalisée supposée à 1 pour tous les pixels.
baseline_reference = torch.ones_like(clean_reference)


Visualisons le spectre propre qui servira de base à toutes les simulations de bruit.

In [ ]:
fig, ax = plt.subplots()
ax.plot(x.numpy(), clean_reference.numpy(), color='black', linewidth=2, label='Spectre propre')
ax.set_xlabel('Position spectrale normalisée')
ax.set_ylabel('Intensité normalisée')
ax.set_title('Spectre synthétique de référence')
ax.legend()
plt.show()


## Génération des échantillons bruités

Nous balayons les valeurs de `noise_level` de 0.0 à 2.0 par pas de 0.1. Pour chaque tranche, cinq exemples indépendants sont créés via `add_noise_variety` en désactivant le pipeline de bruit *legacy* pour ne conserver que le profil complexe demandé.

In [ ]:
noise_types = ['gaussian', 'shot', 'flicker', 'etaloning', 'glitches']
noise_levels = np.round(np.arange(0.0, 2.0 + 1e-6, 0.1), 1)
examples_per_level = 5

# Dictionnaire: type -> niveau -> échantillons (examples_per_level, spectral_length)
noise_samples = {noise_type: {} for noise_type in noise_types}

for t_idx, noise_type in enumerate(noise_types):
    for l_idx, level in enumerate(noise_levels):
        level_key = f"{level:.1f}"
        generator = torch.Generator().manual_seed(10_000 * t_idx + l_idx)
        batch_clean = clean_reference.unsqueeze(0).repeat(examples_per_level, 1)
        batch_baseline = baseline_reference.unsqueeze(0).repeat(examples_per_level, 1)
        noisy = add_noise_variety(
            batch_clean,
            baseline_norm=batch_baseline,
            generator=generator,
            legacy_enabled=False,
            complex={
                'mode': 'replace',
                'noise_type': noise_type,
                'noise_level': float(level),
                'max_rel_to_line': 0.15,
                'clip': (0.0, 1.4),
            },
        )
        noise_samples[noise_type][level_key] = noisy.detach().cpu().numpy()


## Visualisation interactive

Utilisez les contrôles ci-dessous pour sélectionner un type de bruit et une tranche de `noise_level`. Le spectre propre est tracé en noir et les cinq réalisations correspondant à la tranche choisie sont affichées en couleur.

In [ ]:
palette = plt.cm.viridis(np.linspace(0, 1, examples_per_level))

noise_selector = widgets.Dropdown(
    options=[(nt.capitalize(), nt) for nt in noise_types],
    value='gaussian',
    description='Type:',
)

level_selector = widgets.SelectionSlider(
    options=[(f"{lvl:.1f}", f"{lvl:.1f}") for lvl in noise_levels],
    value=f"{noise_levels[0]:.1f}",
    description='Niveau:',
    continuous_update=False,
)

output = widgets.Output()


def plot_examples(noise_type: str, level_key: str) -> None:
    output.clear_output(wait=True)
    examples = noise_samples[noise_type][level_key]

    with output:
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(x.numpy(), clean_reference.numpy(), color='black', linewidth=2, label='Propre')
        for idx in range(examples_per_level):
            ax.plot(x.numpy(), examples[idx], color=palette[idx], alpha=0.8, label=f'Exemple {idx + 1}')
        ax.set_xlabel('Position spectrale normalisée')
        ax.set_ylabel('Intensité normalisée')
        ax.set_title(f"{noise_type.capitalize()} — noise_level = {level_key}")
        ax.set_ylim(0.0, 1.4)
        ax.legend(loc='upper right', ncol=2)
        plt.show()


def on_change(change):
    plot_examples(noise_selector.value, level_selector.value)

noise_selector.observe(on_change, names='value')
level_selector.observe(on_change, names='value')

controls = widgets.HBox([noise_selector, level_selector])
display(controls, output)

plot_examples(noise_selector.value, level_selector.value)


## Statistiques rapides

Le tableau suivant résume l'écart-type moyen du bruit ajouté pour chaque combinaison type/niveau, calculé en soustrayant le spectre propre. Il permet de visualiser globalement la croissance de l'amplitude du bruit lorsque `noise_level` augmente.

In [ ]:
import pandas as pd

rows = []
for noise_type in noise_types:
    for level_key, noisy in noise_samples[noise_type].items():
        residual = noisy - clean_reference.numpy()
        rms = np.sqrt(np.mean(residual**2, axis=1))
        rows.append({'Type': noise_type, 'noise_level': float(level_key), 'RMS moyen': float(rms.mean())})

summary_df = pd.DataFrame(rows).pivot(index='noise_level', columns='Type', values='RMS moyen')
display(summary_df)
